# Guided Project: Analyzing CIA Factbook Data Using SQL

In this project, we'll work with data from the CIA World Factbook, a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like the following:

- `population` — the global population.
- `population_growth` — the annual population growth rate, as a percentage.
- `area` — the total land and water area.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

Begin exploring the data:

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Running query in 'sqlite:///factbook.db'

type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [3]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Running query in 'sqlite:///factbook.db'

id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Here are the descriptions for some of the columns:

- `name` — the name of the country.
- `area` — the country's total area (both land and water).
- `area_land` — the country's land area in square kilometers.
- `area_water` — the country's water area in square kilometers.
- `population` — the country's population.
- `population_growth` — the country's population growth as a percentage.
- `birth_rate` — the country's birth rate, or the number of births per year per 1,000 people.
- `death_rate` — the country's death rate, or the number of death per year per 1,000 people.

### Summary Statistics

Let's start by calculating some summary statistics and look for any outlier countries.

In [4]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth
  FROM facts;

Running query in 'sqlite:///factbook.db'

min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


We see a few interesting things in the summary statistics on the previous screen:

- There's a country with a population of 0
- There's a country with a population of 7256490011 (or more than 7.2 billion people)

Let's use subqueries to zoom in on just these countries without using the specific values.

Let's write a query that returns the countries with the minimum and maximum populations.


In [5]:
%%sql
SELECT *
  FROM facts
 WHERE population = (SELECT MIN(population)
                       FROM facts
                    )


Running query in 'sqlite:///factbook.db'

id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


It appears that the CIA Factbook contains a row for Antarctica which explains the population of zero.

In [6]:
%%sql
SELECT *
  FROM facts
 WHERE population = (SELECT MAX(population)
                       FROM facts
                    )


Running query in 'sqlite:///factbook.db'

id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


The CIA Factbook also contains a row for the `world`. This explains why there is a maximum population listed at over 7.2 billion. 

Given the information above, the summary statisics should be recalculated to exclude the row accounting for the entire world.

In [7]:
%%sql
SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
       MAX(population_growth) AS max_pop_growth
  FROM facts
 WHERE name != 'World'


Running query in 'sqlite:///factbook.db'

min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


### Exploring Average Population and Area

Let's begin by calculating the average value for population and area.

In [8]:
%%sql
SELECT AVG(population) AS avg_pop,
       AVG(area) AS avg_area
  FROM facts
 WHERE name != 'World';

Running query in 'sqlite:///factbook.db'

avg_pop,avg_area
32242666.56846473,555093.546184739


Now we will write a query that finds all countries that meet the following criteria:

- The Population is above average
- The area is below average

In [9]:
%%sql
SELECT name AS country,
       population,
       area
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                      WHERE name != 'World'
                    )
   AND area < (SELECT AVG(area)
                 FROM facts
                WHERE name != 'World'
              );

Running query in 'sqlite:///factbook.db'

country,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Iraq,37056169,438317
Italy,61855120,301340
Japan,126919659,377915
"Korea, South",49115196,99720
Morocco,33322699,446550
Philippines,100998376,300000
Poland,38562189,312685
Spain,48146134,505370


## Additonal Exploration

We will answer the following:
- Which country has the most people? Which country has the highest growth rate?
- Which countries have the highest ratios of water to land? Which countries - have more water than land?
- Which countries will add the most people to their populations next year?
- Which countries have a higher death rate than birth rate?
- Which countries have the highest population/area ratio, and how does it compare to list we found in the previous screen?

### Country with the Largest Population 

In [10]:
%%sql
SELECT name as country, 
       population
  FROM facts
 WHERE population == (SELECT MAX(population)
                        FROM facts
                       WHERE name != 'World'
                     );

Running query in 'sqlite:///factbook.db'

country,population
China,1367485388


### Highest Population Growth

In [11]:
%%sql
SELECT name AS country,
       population_growth
  FROM facts
 WHERE population_growth == (SELECT MAX(population_growth)
                               FROM facts
                              WHERE name != 'World'
                            );

Running query in 'sqlite:///factbook.db'

country,population_growth
South Sudan,4.02




### TOP 5: Countries with the highest water to land ratio

In [12]:
%%sql
SELECT name AS country,
       area_water,
       area_land,
       ROUND(CAST(area_water AS Float)/CAST(area_land AS Float), 2) AS water_to_land
  FROM facts
 ORDER by water_to_land DESC
 LIMIT 5;

Running query in 'sqlite:///factbook.db'

country,area_water,area_land,water_to_land
British Indian Ocean Territory,54340,60,905.67
Virgin Islands,1564,346,4.52
Puerto Rico,4921,8870,0.55
"Bahamas, The",3870,10010,0.39
Guinea-Bissau,8005,28120,0.28


The British Indian Ocean Territory and Virgin Islands have more water than land!

### TOP 5: Highest Populations Next Year

In [13]:
%%sql
SELECT name AS country,
       CAST(population * (1 + (population_growth/100)) AS integer) AS population_next_year
  FROM facts
 WHERE name != 'World' AND name != 'European Union'
 ORDER BY population_next_year DESC
 LIMIT 5;

Running query in 'sqlite:///factbook.db'

country,population_next_year
China,1373639072
India,1266966270
United States,323875541
Indonesia,258348815
Brazil,205832612


### TOP 5: Higher Death Rate than Birth Rate

In [14]:
%%sql
SELECT name AS country,
       death_rate,
       birth_rate
  FROM facts
 WHERE death_rate > birth_rate
 LIMIT 5;

Running query in 'sqlite:///factbook.db'

country,death_rate,birth_rate
Austria,9.42,9.41
Belarus,13.36,10.7
Bosnia and Herzegovina,9.75,8.87
Bulgaria,14.44,8.92
Croatia,12.18,9.45


### TOP 5: Most Densely Populated Countries

NOTE: Since most people live on land, only land area will be used for this calulation. Using the total area, including water area, might distort the results.

In [15]:
%%sql
SELECT name AS country,
       population,
       area_land,
       ROUND(CAST(population/area_land AS Float), 2) AS pop_density
  FROM facts
 ORDER BY pop_density DESC
 LIMIT 5;

Running query in 'sqlite:///factbook.db'

country,population,area_land,pop_density
Macau,592731,28,21168.0
Monaco,30535,2,15267.0
Singapore,5674472,687,8259.0
Hong Kong,7141106,1073,6655.0
Gaza Strip,1869055,360,5191.0


Compare to the list of countries that have an above average population and below average area

In [16]:
%%sql
SELECT name AS country,
       population,
       area_land,
       ROUND(CAST(population/area_land AS Float), 2) AS pop_density
  FROM facts
 WHERE population > (SELECT AVG(population)
                       FROM facts
                      WHERE name != 'World'
                    )
   AND area < (SELECT AVG(area)
                 FROM facts
                WHERE name != 'World'
              )
 ORDER BY pop_density DESC;

Running query in 'sqlite:///factbook.db'

country,population,area_land,pop_density
Bangladesh,168957745,130170,1297.0
"Korea, South",49115196,96920,506.0
Japan,126919659,364485,348.0
Philippines,100998376,298170,338.0
Vietnam,94348835,310070,304.0
United Kingdom,64088222,241930,264.0
Germany,80854408,348672,231.0
Italy,61855120,294140,210.0
Uganda,37101745,197100,188.0
Thailand,67976405,510890,133.0


It is very interesting that the Top 5 most densely populated countries are not on the list of countires with above average population and below average land area. It makes sense, however, when looking at the land area sizes as the most densely populated countries have land areas that are significantly below the average land area size. 